# Fullcustom analog design crashbook

The environment path must be retrieved in order to provide _Magic_ with the technology files.

**Please check them. If they are wrong, you can hardcode the correct ones. If you don't know, let it as it is and check for strange behaviours afterwards 😉**

In [ ]:
import os
from pathlib import Path
CONDA_DIR = os.environ['CONDA_PREFIX']
HOME_DIR = Path.home()
FULLCUSTOM_DIR = os.path.abspath("")

print(f"Anaconda environment absolute path = '{CONDA_DIR}'")
print(f"Home directory absolute path       = '{HOME_DIR}'")
print(f"Fullcustom absolute path           = '{FULLCUSTOM_DIR}'")

## Source files

### Using the _Magic_ VLSI layout tool

[_Magic_](http://opencircuitdesign.com/magic/) cells (`.mag`) are used to store and manage the layouts. They can be edited using _Magic_ commands or using _Magic_'s GUI.

An existing _inverter_ design as well as its dependencies is imported and converted to _Graphic Design System_ (`.gds`) file for viewing and manufacturing and a _SPICE_ (`.spice`) file for simulation:

In [ ]:
%%script magic -dnull -noconsole -rcfile {CONDA_DIR}/share/pdk/sky130A/libs.tech/magic/sky130A.magicrc

load cad/inv/mag/inv.mag

extract all
ext2spice lvs
ext2spice cthresh 0
ext2spice rthresh 0
ext2spice -o cad/inv/spice/inv.spice

gds labels no
gds write cad/inv/gds/inv.gds

### Display the inverter layout

The `.gds` file can be converted to vector graphics for displaying:

In [ ]:
import gdstk
import IPython.display

library = gdstk.read_gds('cad/inv/gds/inv.gds')
top_cells = library.top_level()
top_cells[0].write_svg('cad/inv/svg/inv.svg', scaling=100)
IPython.display.SVG('cad/inv/svg/inv.svg')

## Simulation

### Using _SPICE_ simulation engine

[_PySPICE_](https://pyspice.fabrice-salvaire.fr/releases/v1.4/overview.html) is a _Python_ interface for several _SPICE_ simulation engines. The _SPICE_ engines are used to simulate analog circuits. A top-level test bench circuit must be built:

In [ ]:
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Unit import *

# Declare the top-level circuit
circuit = Circuit('INV')
# Link to technology libraries
circuit.lib(f'{CONDA_DIR}/share/pdk/sky130A/libs.tech/ngspice/sky130.lib.spice', 'tt')

# Import and instantiate the CUT (Circuit Under Test)
circuit.include('cad/inv/spice/inv.spice')
circuit.X('INV', 'inv', 'VDD','VSS','A','X');
circuit.C('','X','VSS','10f')

# Add voltage sources
circuit.V('V_SS', 'VSS', 0, 0)
circuit.V('V_DD', 'VDD', 'VSS', 1.8)

# Add pulse sources
circuit.PulseVoltageSource('A', 'A', 0,
                           initial_value=0@u_V, pulsed_value=1.8@u_V,
                           rise_time=10@u_ps, fall_time=10@u_ps,
                           pulse_width=0.35@u_ns, period=0.8@u_ns, delay_time=0.2@u_ns)

# Create the simulation and simulate
simulator = circuit.simulator()
analysis  = simulator.transient(step_time=10@u_ps, end_time=1@u_ns)

### Plot the waveform

The output waveform can be plotted:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(figsize=(8,5))
ax.grid(visible=True, which='major', axis='both');
ax.grid(visible=True, which='minor', axis='both', linestyle="--");
ax.set_yticks(np.linspace(-1.0,2.5,8,endpoint=True));
ax.set_yticks(np.linspace(-1.0,2.5,36,endpoint=True),minor=True);
ax.set_xticks(np.linspace(0e-12,1e-9,11,endpoint=True));
ax.set_xticks(np.linspace(0e-12,1e-9,21,endpoint=True),minor=True);
ax.set_title('INV Gate - Transient Simulation')
ax.set_xlabel('Time [s]')
ax.set_ylabel('Signal [V]')
ax.plot(analysis.time,analysis.A)
ax.plot(analysis.time,analysis.X)
ax.legend(('Input (A)', 'Output (X)'))
plt.tight_layout()
plt.show()

# To have fun going further...

## Graphical editor

_Magic_ can be started specifying only one file (`.magicrc`) that sets everything up (layers, macros, DRC rules, etc.)! Both a console and a GUI are started. Some functions are only accessible using the command-line interface or using keybindings.

In [ ]:
!magic -rcfile {CONDA_DIR}/share/pdk/sky130A/libs.tech/magic/sky130A.magicrc

## Schematic editor

Of course, the workflow would not be complete without a schematic editor... and an LVS tool. For instance, one can use _Xscheme_:

In [ ]:
!export PDK_ROOT={CONDA_DIR}/share/pdk/; \
xschem --rcfile {CONDA_DIR}/share/pdk/sky130A/libs.tech/xschem/xschemrc

The inverter schematic is available under:
    
```
fullcustom/cad/inv/sch/inv.sch
```

Open it with _xschem_ and generate the netlist by:

 * Selecting _Spice netlist_ in the _Options_ toolbar menu. (This will produce a netlist in a format accepted by the _LVS_ tool.)
 * Selecting _LVS netlist: Top level is a .subckt_ in the _Simulation_ toolbar menu. (This will encapsulate the circuit in a subcircuit. The _LVS_ tool fails otherwose.)
 
Click on _Netlist_ (top right) to generate the netlist. It is generated by default under the user repository:

```
~/.xschem/simulations/<circuit>.spice
```

This location is assumed in the following cells.

## Layout Versus Schematic

The layout netlists takes into account the parasitics. They are modeled as lumped resistances and capacitances but they are reported on the schematic. They must be therefore removed from the schematic's netlist. Comment out the two following lines of the corresponding cell above:

```diff
- ext2spice cthresh 0
- ext2spice rthresh 0
+ #ext2spice cthresh 0
+ #ext2spice rthresh 0
```

Now that both the schematic's and the layout's netlists are written, they can be compare by _netgen_, the LVS tool:

In [ ]:
!netgen lvs \
"{FULLCUSTOM_DIR}/cad/inv/spice/inv.spice inv" \
"{HOME_DIR}/.xschem/simulations/inv.spice inv" \
"{CONDA_DIR}/share/pdk/sky130A/libs.tech/netgen/setup.tcl"

## Some ideas...

Here are a couple ideas to spend a good time exploring those beautiful pieces of software:

 * Characterize an NMOS device or a PMOS device by applying voltage pulses on the different pads.
 * Start _Magic_ on another PDK and design an inverter.
 * Vary the devices' geometry and compare the pull-down and pull-up curves.
 * Copy this notebook to explore the NAND gate cell available in the repository.
 
 **Do not hesitate to open the `.mag` files to see how they are built and to adapt from them!**
 
 > Good luck and read the docs. 😉
 
## More food for the brain

 * http://opencircuitdesign.com/
 * https://skywater-pdk.readthedocs.io

# References

Inspired from:
“Silicon Notebooks.” CHIPS Alliance, Apr. 08, 2023. Accessed: Apr. 10, 2023. [Online]. Available: https://github.com/chipsalliance/silicon-notebooks/blob/b65134a43b01ae31423f7ee87110740b2257ac42/analog-inverter-magic.ipynb (Apache License 2.0)